In [1]:
import math
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, accuracy_score, recall_score
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
from collections import defaultdict
from scipy.sparse import csr_matrix, lil_matrix
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv(r'kz.csv')

In [3]:
def accept_float(value):
    '''
    Read value and replace it with Nan if not float
    '''
    if value is None:
        return np.nan
    try:
        return float(value)
    except ValueError:
        return np.nan
    
def is_float(value):
    '''
    Read value and return True if float, False otherwise 
    '''
    try:
        float(value)
        return True
    except ValueError:
        return False
    
def reject_float(value):
    '''
    Read value and return Nan if it is float
    '''
    if value is None:
        return np.nan
    if is_float(value):
        return np.nan
    else:
        return value
    
def get_month_year(timestamp):
    '''
    Read timestamp and return year-month
    '''
    try:
        # 2020-04-24 11:50:39 UTC
        ft = datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S %Z')
        return datetime.strftime(ft, "%Y-%m")
    except:
        print('Could not process timestamp')
        return None

In [4]:
sales = df.copy()

sales_copy = sales.copy()

display(sales.dtypes)
sales.head()

event_time        object
order_id           int64
product_id         int64
category_id      float64
category_code     object
brand             object
price            float64
user_id          float64
dtype: object

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,NaN,karcher,217.57,1.515916e+18


In [5]:
display(sales.isna().sum()*100/sales.shape[0]) # Процент пустных значений

event_time        0.000000
order_id          0.000000
product_id        0.000000
category_id      16.402148
category_code    23.246521
brand            19.214010
price            16.402148
user_id          78.577387
dtype: float64

In [6]:
sales.user_id.fillna(sales.brand, inplace=True)
sales.user_id = sales.user_id.apply(accept_float)
sales.user_id.fillna(0, inplace=True)

sales.price.fillna(sales.category_code, inplace=True)
sales.price = sales.price.apply(accept_float)
sales.price.fillna(0, inplace=True)

sales.category_id.fillna(0, inplace=True)

sales.brand = sales.brand.apply(reject_float)
sales.brand.fillna('unknown', inplace=True)
print(f'Invalid brand percentage: {sales[sales.brand=="unknown"].shape[0]/sales.shape[0]:.2%}\n')

sales.brand.loc[sales.brand=='none'] = 'unknown'

C:\Users\Илья\AppData\Local\Temp\ipykernel_8876\390519182.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sales.user_id.fillna(sales.brand, inplace=True)
C:\Users\Илья\AppData\Local\Temp\ipykernel_8876\390519182.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['1515915625453675584' '1515915625455624044' '1515915625441101795' ...
 '1515915625514803719' '1515915625514155115' '1515915625514803864']' has dtype incompatible with float64, plea

Invalid brand percentage: 20.68%



C:\Users\Илья\AppData\Local\Temp\ipykernel_8876\390519182.py:15: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  sales.brand.loc[sales.brand=='none'] = 'unknown'


In [7]:
display(sum(sales.category_code.unique()=='none'))

np.int64(0)

In [8]:
sales.category_code = sales.category_code.apply(reject_float)

sales.category_code.fillna('unknown', inplace=True)
print(f'Invalid category_code percentage: {sales[sales.category_code=="unknown"].shape[0]/sales.shape[0]:.2%}\n')

display(sales.isna().sum()/sales.shape[0])

C:\Users\Илья\AppData\Local\Temp\ipykernel_8876\243883942.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  sales.category_code.fillna('unknown', inplace=True)


Invalid category_code percentage: 39.65%



event_time       0.0
order_id         0.0
product_id       0.0
category_id      0.0
category_code    0.0
brand            0.0
price            0.0
user_id          0.0
dtype: float64

In [9]:
sales

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
0,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
1,2020-04-24 11:50:39 UTC,2294359932054536986,1515966223509089906,2.268105e+18,electronics.tablet,samsung,162.01,1.515916e+18
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,unknown,karcher,217.57,1.515916e+18
...,...,...,...,...,...,...,...,...
2633516,2020-11-21 10:10:01 UTC,2388440981134693942,1515966223526602848,2.268105e+18,electronics.smartphone,oppo,138.87,1.515916e+18
2633517,2020-11-21 10:10:13 UTC,2388440981134693943,1515966223509089282,2.268105e+18,electronics.smartphone,apple,418.96,1.515916e+18
2633518,2020-11-21 10:10:30 UTC,2388440981134693944,1515966223509089917,2.268105e+18,appliances.personal.scales,vitek,12.48,1.515916e+18
2633519,2020-11-21 10:10:30 UTC,2388440981134693944,2273948184839454837,2.268105e+18,unknown,moulinex,41.64,1.515916e+18


In [10]:
filtered_sales = sales.groupby('user_id').filter(lambda x: len(x) > 5) # проблема рекомендательных систем в том, что плохо работает с холодными пользователями (0 действий), поэтому отбросим небольшое количество покупок

In [11]:
filtered_sales

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,unknown,karcher,217.57,1.515916e+18
5,2020-04-26 08:45:57 UTC,2295716521449619559,1515966223509261697,2.268105e+18,furniture.kitchen.table,maestro,39.33,1.515916e+18
6,2020-04-26 09:33:47 UTC,2295740594749702229,1515966223509104892,2.268105e+18,electronics.smartphone,apple,1387.01,1.515916e+18
...,...,...,...,...,...,...,...,...
2633504,2020-11-21 09:59:55 UTC,2388440981134693931,2388434452473986696,2.268105e+18,appliances.environment.air_heater,ava,43.96,1.515916e+18
2633508,2020-11-21 10:02:06 UTC,2388440981134693935,2273948303068496095,2.268105e+18,electronics.smartphone,huawei,185.16,1.515916e+18
2633512,2020-11-21 10:06:01 UTC,2388440981134693939,1515966223509090132,2.268105e+18,computers.peripherals.printer,canon,299.98,1.515916e+18
2633514,2020-11-21 10:08:54 UTC,2388440981134693941,1515966223519279912,2.374499e+18,electronics.video.tv,samsung,1736.09,1.515916e+18


In [12]:
filtered_sales.dtypes

event_time        object
order_id           int64
product_id         int64
category_id      float64
category_code     object
brand             object
price            float64
user_id          float64
dtype: object

In [13]:
df_encoded = filtered_sales.copy()

# 1. Кодирование order_id (если есть пропуски, создаем новый порядковый номер)
if 'order_id' in df_encoded.columns:
    if df_encoded['order_id'].isna().any():
        # Если есть пропуски, создаем новые order_id
        df_encoded['order_id'] = range(len(df_encoded))
    else:
        # Кодируем существующие order_id
        order_le = LabelEncoder()
        df_encoded['order_id'] = order_le.fit_transform(df_encoded['order_id'])

# 2. Кодирование product_id
product_le = LabelEncoder()
df_encoded['product_id'] = product_le.fit_transform(df_encoded['product_id'])

# 3. Кодирование category_id
category_id_le = LabelEncoder()
df_encoded['category_id'] = category_id_le.fit_transform(df_encoded['category_id'].astype(str))

# 4. Кодирование category_code (если есть иерархия, можно разбить на части)
if 'category_code' in df_encoded.columns:
    # Заменяем пропуски на 'unknown'
    df_encoded['category_code'] = df_encoded['category_code'].fillna('unknown')
    
    # Если category_code содержит иерархию (например, "electronics.smartphone")
    if df_encoded['category_code'].str.contains('\.').any():
        # Можно создать отдельные признаки для каждого уровня иерархии
        category_parts = df_encoded['category_code'].str.split('.', expand=True)
        for i in range(category_parts.shape[1]):
            le = LabelEncoder()
            df_encoded[f'category_level_{i+1}'] = le.fit_transform(category_parts[i].fillna('unknown'))
    
    # Основное кодирование category_code
    category_code_le = LabelEncoder()
    df_encoded['category_code'] = category_code_le.fit_transform(df_encoded['category_code'])

# 5. Кодирование brand
brand_le = LabelEncoder()
df_encoded['brand'] = brand_le.fit_transform(df_encoded['brand'].fillna('unknown_brand'))

# 6. Кодирование user_id
user_le = LabelEncoder()
df_encoded['user_id'] = user_le.fit_transform(df_encoded['user_id'])

# 7. Преобразование price * 100 в int
if 'price' in df_encoded.columns:
    # Умножаем на 100 и преобразуем в int
    df_encoded['price_int'] = (df_encoded['price'] * 100).astype(int)
    
    # Удаляем исходную колонку price если нужно
    # df_encoded = df_encoded.drop('price', axis=1)

# 8. Преобразование event_time в timestamp
if 'event_time' in df_encoded.columns:
    # Преобразуем в datetime если еще не преобразовано
    df_encoded['event_time'] = pd.to_datetime(df_encoded['event_time'])
    
    # Создаем timestamp в секундах (или миллисекундах)
    df_encoded['timestamp'] = df_encoded['event_time'].astype('int64') // 10**9  # секунды
    # Или для миллисекунд:
    # df_encoded['timestamp_ms'] = df_encoded['event_time'].astype('int64') // 10**6
    
    # Дополнительно: извлекаем особенности времени
    df_encoded['hour'] = df_encoded['event_time'].dt.hour
    df_encoded['day_of_week'] = df_encoded['event_time'].dt.dayofweek
    df_encoded['month'] = df_encoded['event_time'].dt.month

# Проверяем результаты
print("Типы данных после преобразования:")
print(df_encoded.dtypes)
print("\nПервые 5 строк:")
print(df_encoded.head())
print(f"\nРазмер датафрейма: {df_encoded.shape}")

# Сохраняем mapping для обратного преобразования если нужно
mappings = {
    'product_le': product_le,
    'category_id_le': category_id_le,
    'brand_le': brand_le,
    'user_le': user_le
}

if 'category_code' in df_encoded.columns:
    mappings['category_code_le'] = category_code_le

<>:27: SyntaxWarning: invalid escape sequence '\.'
<>:27: SyntaxWarning: invalid escape sequence '\.'
C:\Users\Илья\AppData\Local\Temp\ipykernel_8876\661876700.py:27: SyntaxWarning: invalid escape sequence '\.'
  if df_encoded['category_code'].str.contains('\.').any():


Типы данных после преобразования:
event_time          datetime64[ns, UTC]
order_id                          int64
product_id                        int64
category_id                       int64
category_code                     int64
brand                             int64
price                           float64
user_id                           int64
category_level_1                  int64
category_level_2                  int64
category_level_3                  int64
price_int                         int64
timestamp                         int64
hour                              int32
day_of_week                       int32
month                             int32
dtype: object

Первые 5 строк:
                 event_time  order_id  product_id  category_id  category_code  \
2 2020-04-24 14:37:43+00:00         0       19567          343             80   
3 2020-04-24 14:37:43+00:00         0       19567          343             80   
4 2020-04-24 19:16:21+00:00         1       19486   

In [14]:
filtered_sales

,event_time,order_id,product_id,category_id,category_code,brand,price,user_id
2,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
3,2020-04-24 14:37:43 UTC,2294444024058086220,2273948319057183658,2.268105e+18,electronics.audio.headphone,huawei,77.52,1.515916e+18
4,2020-04-24 19:16:21 UTC,2294584263154074236,2273948316817424439,2.268105e+18,unknown,karcher,217.57,1.515916e+18
5,2020-04-26 08:45:57 UTC,2295716521449619559,1515966223509261697,2.268105e+18,furniture.kitchen.table,maestro,39.33,1.515916e+18
6,2020-04-26 09:33:47 UTC,2295740594749702229,1515966223509104892,2.268105e+18,electronics.smartphone,apple,1387.01,1.515916e+18
...,...,...,...,...,...,...,...,...
2633504,2020-11-21 09:59:55 UTC,2388440981134693931,2388434452473986696,2.268105e+18,appliances.environment.air_heater,ava,43.96,1.515916e+18
2633508,2020-11-21 10:02:06 UTC,2388440981134693935,2273948303068496095,2.268105e+18,electronics.smartphone,huawei,185.16,1.515916e+18
2633512,2020-11-21 10:06:01 UTC,2388440981134693939,1515966223509090132,2.268105e+18,computers.peripherals.printer,canon,299.98,1.515916e+18
2633514,2020-11-21 10:08:54 UTC,2388440981134693941,1515966223519279912,2.374499e+18,electronics.video.tv,samsung,1736.09,1.515916e+18


In [15]:
filtered_sales = df_encoded

repeat_customers = filtered_sales.groupby('user_id').size()
k = int(repeat_customers.median())
k = max(3, min(k, 10))  # Ограничиваем k от 3 до 10

In [16]:
k = 5

In [17]:
filtered_sales = filtered_sales.drop(['event_time', 'price', 'category_level_1', 'category_level_2', 'category_level_3', 'hour', 'day_of_week', 'month'], axis=1)

In [18]:
train_data = filtered_sales[sales['event_time'] < '2020-09-01']
test_data = filtered_sales[sales['event_time'] >= '2020-09-01']

C:\Users\Илья\AppData\Local\Temp\ipykernel_8876\75154612.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_data = filtered_sales[sales['event_time'] < '2020-09-01']
C:\Users\Илья\AppData\Local\Temp\ipykernel_8876\75154612.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  test_data = filtered_sales[sales['event_time'] >= '2020-09-01']


In [19]:
train_data.shape, test_data.shape

((1955777, 8), (513548, 8))

In [20]:
train_data

,order_id,product_id,category_id,category_code,brand,user_id,price_int,timestamp
2,0,19567,343,80,350,3146,7752,1587739063
3,0,19567,343,80,350,3146,7752,1587739063
4,1,19486,603,122,404,1705,21757,1587755781
5,2,4669,423,98,465,3798,3933,1587890757
6,3,1158,327,87,40,3397,138701,1587893627
...,...,...,...,...,...,...,...,...
2439301,1195942,101,327,87,40,16719,92567,1598867463
2471364,1212054,28,327,87,663,17046,30089,1595067629
2534091,1243009,7331,126,98,568,17609,6942,1592640888
2534092,1243009,2328,128,53,568,17609,1155,1592640888


In [21]:
def naive_popularity_based(train_data, k):
    popular_items = train_data['product_id'].value_counts().head(k).index.tolist()
    return popular_items

# Наивный алгоритм 2: Любимый товар клиента + (k-1) самых популярных
def naive_hybrid_based(train_data, k):
    # Самые популярные товары
    popular_items = train_data['product_id'].value_counts().head(k).index.tolist()

    # Любимый товар каждого клиента (самый частый)
    user_favorites = train_data.groupby('user_id')['product_id'].apply(
        lambda x: x.value_counts().index[0] if len(x.value_counts()) > 0 else popular_items[0]
    ).to_dict()

    recommendations = {}
    for user in set(train_data['user_id'].unique()) | set(test_data['user_id'].unique()):
        if user in user_favorites:
            favorite = user_favorites[user]
            # Убедимся, что favorite не входит в popular_items
            user_rec = [favorite] + [item for item in popular_items if item != favorite][:k-1]
        else:
            # Если пользователя нет в тренировочных данных, используем популярные товары
            user_rec = popular_items[:k]
        recommendations[user] = user_rec[:k]

    return recommendations

def evaluate_recommendations(recommendations, test_data, k):
    precisions = []
    recalls = []
    f1_scores = []

    # Группируем тестовые данные по пользователям
    test_user_items = test_data.groupby('user_id')['product_id'].apply(set).to_dict()

    for user, actual_items in test_user_items.items():
        if user in recommendations:
            recommended_items = set(recommendations[user][:k])
        else:
            # Если пользователя нет в рекомендациях, пропускаем
            continue

        if len(recommended_items) == 0:
            precision = recall = f1 = 0
        else:
            # Вычисляем precision и recall
            true_positives = len(recommended_items & actual_items)
            precision = true_positives / len(recommended_items)
            recall = true_positives / len(actual_items) if len(actual_items) > 0 else 0

            # Вычисляем F1-score
            if precision + recall > 0:
                f1 = 2 * (precision * recall) / (precision + recall)
            else:
                f1 = 0

        precisions.append(precision)
        recalls.append(recall)
        f1_scores.append(f1)

    # Усредняем метрики по всем пользователям
    avg_precision = np.mean(precisions) if precisions else 0
    avg_recall = np.mean(recalls) if recalls else 0
    avg_f1 = np.mean(f1_scores) if f1_scores else 0

    return avg_precision, avg_recall, avg_f1


In [22]:
popular_items = naive_popularity_based(train_data, k)
popular_recommendations = {user: popular_items for user in test_data['user_id'].unique()}
hybrid_recommendations = naive_hybrid_based(train_data, k)


precision_pop, recall_pop, f1_pop  = evaluate_recommendations(popular_recommendations, test_data, 3)

precision_hyb, recall_hyb, f1_hyb = evaluate_recommendations(hybrid_recommendations, test_data, k)
print(f'Popularity Based - Precision: {precision_pop:.4f}, Recall: {recall_pop:.4f}, F1: {f1_pop:.4f}')
print(f'Hybrid Based - Precision: {precision_hyb:.4f}, Recall: {recall_hyb:.4f}, F1: {f1_hyb:.4f}')

Popularity Based - Precision: 0.0476, Recall: 0.0067, F1: 0.0094
Hybrid Based - Precision: 0.0450, Recall: 0.0179, F1: 0.0153


In [27]:
train_data

,order_id,product_id,category_id,category_code,brand,user_id,price_int,timestamp
2,0,19567,343,80,350,3146,7752,1587739063
3,0,19567,343,80,350,3146,7752,1587739063
4,1,19486,603,122,404,1705,21757,1587755781
5,2,4669,423,98,465,3798,3933,1587890757
6,3,1158,327,87,40,3397,138701,1587893627
...,...,...,...,...,...,...,...,...
2439301,1195942,101,327,87,40,16719,92567,1598867463
2471364,1212054,28,327,87,663,17046,30089,1595067629
2534091,1243009,7331,126,98,568,17609,6942,1592640888
2534092,1243009,2328,128,53,568,17609,1155,1592640888


In [23]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
from collections import defaultdict

# ==== 1. Построение user-item матрицы ====
def build_interaction_matrix(train_data):
    users = train_data['user_id'].unique()
    items = train_data['product_id'].unique()
    user_map = {u: i for i, u in enumerate(users)}
    item_map = {p: i for i, p in enumerate(items)}
    inv_item_map = {i: p for p, i in item_map.items()}

    rows = train_data['user_id'].map(user_map)
    cols = train_data['product_id'].map(item_map)
    data = np.ones(len(train_data), dtype=np.float32)

    interaction_matrix = csr_matrix((data, (rows, cols)), shape=(len(users), len(items)))
    return interaction_matrix, user_map, item_map, inv_item_map

# ==== 2. Вычисление сходства между товарами ====
def compute_item_similarity(interaction_matrix):
    print("Вычисление косинусного сходства между товарами...")
    similarity = cosine_similarity(interaction_matrix.T, dense_output=False)
    return similarity

# ==== 3. Генерация top-K рекомендаций ====
def recommend_item_cf(train_data, test_data, k=5):
    interaction_matrix, user_map, item_map, inv_item_map = build_interaction_matrix(train_data)
    item_similarity = compute_item_similarity(interaction_matrix)

    recommendations = {}
    all_users = set(train_data['user_id']).union(set(test_data['user_id']))
    popular_items = train_data['product_id'].value_counts().head(k).index.tolist()

    print("Генерация рекомендаций пользователям...")
    for user in tqdm(all_users, desc="Рекомендации", unit="user"):
        if user not in user_map:
            # Холодный старт
            recommendations[user] = popular_items
            continue

        uid = user_map[user]
        user_interactions = interaction_matrix.getrow(uid).toarray().ravel()

        # Оценка для каждого товара = сумма сходств с купленными товарами
        scores = item_similarity.dot(user_interactions)
        scores[user_interactions > 0] = -np.inf  # исключаем уже купленные

        top_items = np.argsort(-scores)[:k]
        recommendations[user] = [inv_item_map[i] for i in top_items]

    return recommendations

# ==== 4. Метрики Precision/Recall/F1 ====
def evaluate_recommendations(recommendations, test_data, k=5):
    precision_list, recall_list, f1_list = [], [], []
    true_user_items = test_data.groupby('user_id')['product_id'].apply(set).to_dict()
    print("Оценка качества рекомендаций...")
    for uid, rec_items in tqdm(recommendations.items(), desc="Оценка", unit="user"):
        true_items = true_user_items.get(uid, set())
        if not true_items:
            continue
        hits = len(set(rec_items) & true_items)
        precision = hits / k
        recall = hits / len(true_items)
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        precision_list.append(precision)
        recall_list.append(recall)
        f1_list.append(f1)
    return np.mean(precision_list), np.mean(recall_list), np.mean(f1_list)

# ==== 5. Пример запуска ====
k = 5
recommendations = recommend_item_cf(train_data, test_data, k=k)
precision, recall, f1 = evaluate_recommendations(recommendations, test_data, k=k)

print(f"\nItem-based CF - Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

Вычисление косинусного сходства между товарами...
Генерация рекомендаций пользователям...


Рекомендации: 100%|██████████| 17963/17963 [1:13:47<00:00,  4.06user/s] 


Оценка качества рекомендаций...


Оценка: 100%|██████████| 17963/17963 [00:00<00:00, 535950.28user/s]


Item-based CF - Precision: 0.0351, Recall: 0.0065, F1: 0.0077
